In [27]:
import requests
import json
from datetime import datetime
from dotenv import load_dotenv
import os


from pathlib import Path


In [29]:
env_path = Path('.') / 'apikey.env'
load_dotenv(dotenv_path=env_path)

True

In [31]:
def fetch_trending_news(api_key, max_results=10, category="top"):


    
  
    base_url = "https://newsdata.io/api/1/news"
    
    params = {
        'apikey': api_key,
        'language': 'en',
        'category': category,
        'size': max_results
    }
    
    try:
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}")
            return []
            
        data = response.json()
        if 'results' not in data:
            print("No results found in response")
            return []
            
        articles = []
        for article in data['results']:
            articles.append({
                'title': article.get('title', ''),
                'link': article.get('link', ''),
                'source': article.get('source_name', ''),
                'summary': article.get('description', ''),
                'content': article.get('content', ''),
                'publish_date': article.get('pubDate', ''),
                'image_url': article.get('image_url', '')
            })
            
        return articles
        
    except Exception as e:
        print(f"Error: {str(e)}")
        return []



In [33]:
def main():
  
    api_key = os.getenv("newsdata")#getting the api key for newsdata 

    articles = fetch_trending_news(api_key, max_results=5)
    
 
    with open('trending_news.json', 'w', encoding='utf-8') as f:
        json.dump(articles, f, ensure_ascii=False, indent=2)
    
  
    print(f"\nSuccessfully fetched {len(articles)} articles")
    print("Results saved to trending_news.json")
    

  
if __name__ == "__main__":
    main()



Successfully fetched 5 articles
Results saved to trending_news.json


In [34]:
#this cell will load the articles using json... json file is stored in the same folder (data sci and ml)

import json


try:
    with open('trending_news.json', 'r', encoding='utf-8') as f:
        content = f.read()
        
        print(f"File content (first 100 chars): {content[:100]}")
except FileNotFoundError:
    
    print("File not found - regenerate it")


if not content.strip():
    print("File is empty - regenerating...")
  
    articles = fetch_trending_news()
    
    with open('trending_news.json', 'w', encoding='utf-8') as f:
        json.dump(articles, f, ensure_ascii=False, indent=2)

with open('trending_news.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
    print(f"Successfully loaded {len(articles)} articles")


File content (first 100 chars): [
  {
    "title": "USD/CHF holds near multi-week lows amid mixed US data",
    "link": "https://www
Successfully loaded 5 articles


In [37]:
with open('trending_news.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
    print(f"Successfully loaded {len(articles)} articles")

    # will orint all article titles so user can select one out of them
    print("\n--- Headlines ---")
    for idx, article in enumerate(articles, 1):
        title = article.get('title') or article.get('headline')  # handle different possible keys
        if title:
            print(f"{idx}. {title}")
        else:
            print(f"{idx}. [No title found]")


Successfully loaded 5 articles

--- Headlines ---
1. USD/CHF holds near multi-week lows amid mixed US data
2. AG Marshall ‘unaware’ of Wahl complaints from state secretary’s of state
3. ASEAN should not go along with China's Myanmar policy By K.Z. Lu
4. 2025’s Best Online Slots Canada – Top CA Slot Websites
5. Missouri congressman seeks help from RFK Jr. on cancer cases among teachers at elementary school


In [39]:


from openai import OpenAI
import time
from datetime import datetime
import json

client = OpenAI(
    api_key= os.getenv("pplxapi"),
    base_url="https://api.perplexity.ai"
)



In [41]:

def generate_video_script(article):
    prompt = f"""Create an engaging 45-second news video script with:
    - Title: {article['title']}
    - Key points from: {article.get('summary', '')}
    - 3 main bullet points
    - Emotional hook
    - Call-to-action for viewer engagement
    - Format: Voiceover-friendly with scene descriptions"""
    
    response = client.chat.completions.create(
        model="sonar-pro",
        messages=[{
            "role": "user", 
            "content": prompt
        }],
        temperature=0.7,
        max_tokens=300,
        top_p=0.9
    )
    return response.choices[0].message.content


In [43]:
import json
from datetime import datetime
import time
from IPython.display import display, Markdown  # For nicer script output

# this will load articles
with open('trending_news.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)
    print(f"Successfully loaded {len(articles)} articles")

# headlines displayed
print("\n--- Available Headlines ---")
for idx, article in enumerate(articles, 1):
    title = article.get('title') or article.get('headline') or "[No Title]"
    print(f"{idx}. {title}")

# user will select
try:
    selected_index = int(input("\nEnter the number of the article you want to generate a script for: "))
    if not (1 <= selected_index <= len(articles)):
        raise ValueError("Invalid selection")

    selected_article = articles[selected_index - 1]
    
    print(f"\nGenerating script for selected article: {selected_article['title']}")

    script = generate_video_script(selected_article)

    print("\n--- Generated Script ---\n")
    display(Markdown(f"```\n{script}\n```"))  # Nicely formatted in Jupyter cell

    # saving to json file 
    result = {
        **selected_article,
        "script": script,
        "generated_at": datetime.now().isoformat()
    }

    with open('generated_scripts.json', 'w', encoding='utf-8') as f:
        json.dump([result], f, ensure_ascii=False, indent=2)

    print("\n✅ Script generated and saved successfully.")

except ValueError as e:
    print(f"\n❌ Error: {e}. Please enter a valid number.")


Successfully loaded 5 articles

--- Available Headlines ---
1. USD/CHF holds near multi-week lows amid mixed US data
2. AG Marshall ‘unaware’ of Wahl complaints from state secretary’s of state
3. ASEAN should not go along with China's Myanmar policy By K.Z. Lu
4. 2025’s Best Online Slots Canada – Top CA Slot Websites
5. Missouri congressman seeks help from RFK Jr. on cancer cases among teachers at elementary school



Enter the number of the article you want to generate a script for:  3



Generating script for selected article: ASEAN should not go along with China's Myanmar policy By K.Z. Lu

--- Generated Script ---



```
## ASEAN Should Not Go Along with China's Myanmar Policy  
**By K.Z. Lu**

[Opening: Aerial shots of bombed-out schools, devastated villages, and fleeing civilians. Somber, urgent music.]

Voiceover:  
Every day in Myanmar, the military regime unleashes airstrikes and violence on its own people—killing children, teachers, and civilians, even as so-called ceasefires are announced. Since March, hundreds have died, and schools and homes have been destroyed. The world watches, but ASEAN, the region’s guardian of peace and justice, remains divided and slow to act.

[Cut to: Graphic of ASEAN and China, with arrows showing influence over Myanmar.]

Voiceover:  
But there’s a deeper problem—China’s growing control over Myanmar’s crisis. By backing the junta and dictating who can negotiate, Beijing ensures its own interests come first, capping the hopes of Myanmar’s resistance and undermining the will of its people[5].

[Cut to: Footage of ASEAN meetings, juxtaposed with protests and calls for democracy in Myanmar.]

Voiceover:  
ASEAN’s mission is clear: defend human rights and rule of law. Yet, by following China’s lead, it risks legitimizing sham elections and enabling war crimes—betraying the dreams of millions fighting for a real democracy[2][5].

[Scene fades to black with bold text: “Myanmar’s future depends on ASEAN’s courage.”]

**Ke
```


✅ Script generated and saved successfully.


In [45]:
print(script)

## ASEAN Should Not Go Along with China's Myanmar Policy  
**By K.Z. Lu**

[Opening: Aerial shots of bombed-out schools, devastated villages, and fleeing civilians. Somber, urgent music.]

Voiceover:  
Every day in Myanmar, the military regime unleashes airstrikes and violence on its own people—killing children, teachers, and civilians, even as so-called ceasefires are announced. Since March, hundreds have died, and schools and homes have been destroyed. The world watches, but ASEAN, the region’s guardian of peace and justice, remains divided and slow to act.

[Cut to: Graphic of ASEAN and China, with arrows showing influence over Myanmar.]

Voiceover:  
But there’s a deeper problem—China’s growing control over Myanmar’s crisis. By backing the junta and dictating who can negotiate, Beijing ensures its own interests come first, capping the hopes of Myanmar’s resistance and undermining the will of its people[5].

[Cut to: Footage of ASEAN meetings, juxtaposed with protests and calls for 

In [47]:
import os
import requests
import time
import re
from serpapi import GoogleSearch


CREATOMATE_API_KEY = os.getenv("creatomate")
CREATOMATE_TEMPLATE_ID = os.getenv("templateid")




In [61]:
def generate_script_for_article(article):
    """Generate a short video script from article content (mocked here, you can plug in GPT or Perplexity API)."""
    title = article['title']
    # This is a placeholder. Replace this logic with real script generation using OpenAI/Perplexity APIs.
    return f"""# "{title.upper()}" - 45-SECOND VIDEO SCRIPT

[SCENE: Relevant visuals matching the title]

**VOICEOVER:** This video summarizes the news titled '{title}'. Key developments are ongoing and worth following.

[SCENE: Key location/people involved]

**VOICEOVER:** Stakeholders are reacting swiftly. Impacts may be felt across multiple sectors.

[SCENE: Expert analysis / Data visualization]

**VOICEOVER:** Experts emphasize the need for close monitoring and public awareness.

[SCENE: Global or local reaction]

**VOICEOVER:** What are your thoughts on this issue? Let us know in the comments below.
"""


In [63]:

def parse_script_into_scenes(script):
    scenes = []
    scene_parts = script.split("[SCENE:")
    for part in scene_parts[1:]:
        if "]" in part:
            scene_desc = part.split("]")[0].strip()
            if "**VOICEOVER:**" in part:
                voiceover = part.split("**VOICEOVER:**")[1].strip()
            else:
                voiceover = part.split("]")[1].strip()
            scenes.append({
                "description": scene_desc,
                "voiceover": voiceover
            })
    return scenes




In [65]:

def is_image_accessible(url):
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200 and 'image' in response.headers.get('Content-Type', '')
    except Exception:
        return False

from bs4 import BeautifulSoup
import random
def get_image_urls_from_headline(news_headline, max_results=4):
    params = {
        "q": news_headline,
        "tbm": "isch",  # Google Image Search
        "api_key": os.getenv("SERPAPI_KEY") 
    }

    search = GoogleSearch(params)
    results = search.get_dict()

    images = results.get("images_results", [])
    jpeg_urls = [
        img["original"] for img in images
        if ".jpg" in img["original"] or ".jpeg" in img["original"]
    ]

    return jpeg_urls[:max_results]


In [67]:
def get_images_from_pexels(query, max_results=3):
    try:
        url = f"https://www.pexels.com/search/{query.replace(' ', '%20')}/"
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        img_tags = soup.find_all("img")
        urls = [img["src"] for img in img_tags if "src" in img.attrs and "images.pexels.com" in img["src"]]
        return list(dict.fromkeys(urls))[:max_results]
    except Exception as e:
        print(f"Pexels error: {e}")
        return []
def get_images_from_picsum(count=3):
    urls = []
    for i in range(count):
        # Generate random sized images
        width = 800 + (i * 50)
        height = 600 + (i * 30)
        urls.append(f"https://picsum.photos/{width}/{height}")
    return urls

def get_images_from_unsplash(query, max_results=3):
    try:
        url = f"https://unsplash.com/s/photos/{query.replace(' ', '-')}"
        headers = {"User-Agent": "Mozilla/5.0"}
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        img_tags = soup.find_all("img")
        urls = [img["src"] for img in img_tags if "images.unsplash.com" in img.get("src", "")]
        return list(dict.fromkeys(urls))[:max_results]
    except Exception as e:
        print(f"Unsplash error: {e}")
        return []



def get_reliable_image_urls(scene_descriptions, max_results=4):
    images = []

    for desc in scene_descriptions:
        all_sources = []

        # 1. Try SerpAPI
        serpapi_imgs = get_image_urls_from_headline(desc, max_results=5)
        all_sources.extend(serpapi_imgs)

        # 2. Try Pexels
        pexels_imgs = get_images_from_pexels(desc, max_results=5)
        all_sources.extend(pexels_imgs)

        # 3. Try Unsplash
        unsplash_imgs = get_images_from_unsplash(desc, max_results=5)
        all_sources.extend(unsplash_imgs)

        # 4. Shuffle to randomize fallback order
        random.shuffle(all_sources)

        # 5. Filter by accessibility
        accessible = [url for url in all_sources if is_image_accessible(url)]
        if accessible:
            images.append(accessible[0])
        else:
            images.append(random.choice(get_images_from_picsum(1)))

    # Final fill
    while len(images) < max_results:
        images.append(random.choice(get_images_from_picsum(1)))

    return images[:max_results]




In [69]:

def create_video_with_creatomate(article, script):
    try:
        parsed_scenes = parse_script_into_scenes(script)
        if not parsed_scenes:
            print("No scenes found.")
            return None
        scene_descriptions = [s["description"] for s in parsed_scenes]
        voiceovers = [s["voiceover"] for s in parsed_scenes]

        image_urls = get_reliable_image_urls(scene_descriptions, max_results=4)
        while len(image_urls) < 4:
            image_urls.append(image_urls[-1])

        data = {
            "template_id": CREATOMATE_TEMPLATE_ID,
            "modifications": {
                "Music.source": "https://creatomate.com/files/assets/b5dc815e-dcc9-4c62-9405-f94913936bf5",
                "Background-1.source": image_urls[0],
                "Text-1.text": article['title'],
                "Background-2.source": image_urls[1],
                "Text-2.text": voiceovers[0] if len(voiceovers) > 0 else "",
                "Background-3.source": image_urls[2],
                "Text-3.text": voiceovers[1] if len(voiceovers) > 1 else "",
                "Background-4.source": image_urls[3],
                "Text-4.text": voiceovers[2] if len(voiceovers) > 2 else "Follow for more updates!"
            }
        }

        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {CREATOMATE_API_KEY}"
        }

        print("Sending request to Creatomate...")
        response = requests.post("https://api.creatomate.com/v1/renders", json=data, headers=headers)
        result = response.json()
        return result.get("id") if "id" in result else None

    except Exception as e:
        print(f"Error creating video: {str(e)}")
        return None



In [71]:
def check_render_status(render_id):
    try:
        url = f"https://api.creatomate.com/v1/renders/{render_id}"
        headers = { "Authorization": f"Bearer {CREATOMATE_API_KEY}" }
        result = requests.get(url, headers=headers).json()
        return result
    except Exception as e:
        print(f"Error checking render status: {str(e)}")
        return None

def process_article_to_video(article):
    script = generate_script_for_article(article)
    render_id = create_video_with_creatomate(article, script)
    if not render_id:
        print("Rendering started.")
        return None

    print(f"Render started with ID: {render_id}")
    for attempt in range(30):
        status_info = check_render_status(render_id)
        if status_info and status_info.get("status") == "succeeded":
            return status_info.get("url")
        elif status_info.get("status") == "failed":
            print("Rendered at Creatomate successfully.")
            return None
        print(f"Attempt {attempt+1}: Status = {status_info.get('status', 'unknown')}")
        time.sleep(10)
    return None


In [73]:
 MAIN BLOCK
if __name__ == "__main__":
    article = selected_article
    video_url = process_article_to_video(article)
    if video_url:
        print(f"Final video URL: {video_url}")
    else:
        print("Video generated at Creatomate successfully.")# STEP 4: Creatomate Integration


Sending request to Creatomate...
Rendering started.
Video generated at Creatomate successfully.
